<a href="https://colab.research.google.com/github/jishnujp/US_senate_roll_call/blob/main/voteSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vote Summary

## Roll Call Vote 117th Congress - 1st Session
### **Subject**
On the Nomination (Confirmation: Tiffany P. Cunningham, of Illinois, to be United States Circuit Judge for the Federal Circuit.)

### Votes
- YEAs - 63
- NAYs - 33
- Not Voting - 4

link - https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00267.htm



## Scraping

In [12]:
## We have the xml link, which make scraping easier.
import pandas as pd
from bs4 import BeautifulSoup
from requests import get

link = "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00267.xml"
members = BeautifulSoup(get(link).text,'xml').find('members')
votes = pd.DataFrame(columns = ['first_name','last_name','party','state','vote_cast'])
for i,m in enumerate(members.find_all('member')):
    votes.loc[i] = [m.first_name.text, m.last_name.text, m.party.text, m.state.text, m.vote_cast.text]
display(votes.head())
votes.to_csv('roll_call_votes.csv')

,first_name,last_name,party,state,vote_cast
0,Tammy,Baldwin,D,WI,Yea
1,John,Barrasso,R,WY,Nay
2,Michael,Bennet,D,CO,Yea
3,Marsha,Blackburn,R,TN,Nay
4,Richard,Blumenthal,D,CT,Yea


## Exploring

### Basic

In [130]:
party_name ,summary= {'R':'Republican','D':'Democrat','I':'Independent'},{}

for vote,group in votes.groupby('vote_cast'):
    summary[vote] = {}
    for party,sub_group in group.groupby('party'):
        summary[vote][party_name[party]] = len(sub_group)
        
summary = pd.DataFrame(summary).fillna(0).astype(int)
summary.insert(0,'Yea',summary.pop('Yea'))
summary.loc['Total'] = summary.sum()
summary['Total'] = summary.sum(axis = 1)

display(summary)


,Yea,Nay,Not Voting,Total
Republican,15,33,2,50
Democrat,46,0,2,48
Independent,2,0,0,2
Total,63,33,4,100


In [161]:
# What is the votes ratio of Democrats' major decision to Republicans' major decision ?
maj_ratio = round(summary.loc['Democrat'][:-1].max()/summary.loc['Republican'][:-1].max(),2)
print(f"Votes ratio of Democrats' major decision to Republicans' major decision is {maj_ratio}")

Votes ratio of Democrats' major decision to Republicans' major decision is 1.39


In [160]:
# What is the highest absolute difference in percentage between yea and nay senators among the different parties?
print("---------Normalized Summary-----------")
summary_norm = summary.iloc[:,:-1].div(summary.Total,axis = 0)*100
summary_norm['y_n_diff'] = abs(summary_norm.Yea - summary_norm.Nay)
display(summary_norm.round(1))
R_diff,D_diff = summary_norm.loc['Republican'].y_n_diff, summary_norm.loc['Democrat'].y_n_diff
print(f"The difference in percentage between yea and nay among Republicans is {R_diff}")
print(f"The difference in percentage between yea and nay among Democrats is {round(D_diff,2)}")
print(f"highest absolute difference in percentage between yea and nay among the different parties is {round(max(R_diff,D_diff),2)}")

---------Normalized Summary-----------


,Yea,Nay,Not Voting,y_n_diff
Republican,30.0,66.0,4.0,36.0
Democrat,95.8,0.0,4.2,95.8
Independent,100.0,0.0,0.0,100.0
Total,63.0,33.0,4.0,30.0


The difference in percentage between yea and nay among Republicans is 36.0
The difference in percentage between yea and nay among Democrats is 95.83
highest absolute difference in percentage between yea and nay among the different parties is 95.83


### State wise

In [91]:
# How many states have both Republican senators?                   ## RR
# How many states have both Democrat senators?                     ## DD
# How many states have both Republican and Democrat senators?      ## RD
# How many states voted all Yay?                                   ## YY
# How many states voted all Nay?                                   ## NN
# How many states had at least 1 non-voting senator?               ## NV

RR,DD,RD,YY,NN,NV = 0,0,0,0,0,0
for state,group in votes[['state','party','vote_cast']].groupby('state'):
    party,vote = set(group.party),set(group.vote_cast)
    
    ## ---------State wise senetor party counts----------
    if party == {'R'}:
        RR += 1
    elif party =={'D'}:
        DD += 1
    elif party == {'R','D'}:
        RD += 1
        
    ## ---------State wise vote counts--------
    if vote == {'Yea'}:
        YY +=1
    elif vote == {'Nay'}:
        NN += 1
    elif 'Not Voting' in vote:
        NV += 1
        
print(f"{RR} states have both Republican senators")
print(f"{DD} states have both Democrat senators")
print(f"{RD} states have both Republican and Democrat senators")
print(f"{YY} states voted all Yay")
print(f"{NN} states voted all Nay")
print(f"{NV} states had at least 1 non-voting senator")

22 states have both Republican senators
21 states have both Democrat senators
5 states have both Republican and Democrat senators
26 states voted all Yay
12 states voted all Nay
4 states had at least 1 non-voting senator


### Funny

In [178]:
# How many Senators have their names starting with W?
names = votes.copy()
names.insert(0,'name',names.first_name + " " + names.last_name)
names = names.drop(['first_name','last_name','party','state'],axis = 1)
display(names.loc[names.name.str.contains('W')].reset_index(drop = True))

# What's the total number of Yea votes from all senators whose name starts with "B"?
display(names.loc[(names.name.str.contains('B')) & (names.vote_cast == 'Yea')].reset_index(drop = True))

,name,vote_cast
0,Mark Warner,Yea
1,Raphael Warnock,Yea
2,Elizabeth Warren,Yea
3,Sheldon Whitehouse,Yea
4,Roger Wicker,Nay
5,Ron Wyden,Yea


,name,vote_cast
0,Tammy Baldwin,Yea
1,Michael Bennet,Yea
2,Richard Blumenthal,Yea
3,Sherrod Brown,Yea
4,Richard Burr,Yea
5,Ben Cardin,Yea
6,Bob Casey,Yea
7,Ben Lujan,Yea
8,Bernard Sanders,Yea
9,Brian Schatz,Yea


## Plots